In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train = pd.read_csv('/content/drive/MyDrive/데이콘/퇴근시간버스승차예측/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/데이콘/퇴근시간버스승차예측/data/test.csv')

In [4]:
train.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,6.0,26.0,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


data 컬럼을 판다스의 datetime 변수형으로 변환.

In [5]:
train['date'] = pd.to_datetime(train['date'])
print(train['date'])

0        2019-09-01
1        2019-09-01
2        2019-09-01
3        2019-09-01
4        2019-09-01
            ...    
415418   2019-09-30
415419   2019-09-30
415420   2019-09-30
415421   2019-09-30
415422   2019-09-30
Name: date, Length: 415423, dtype: datetime64[ns]


date 컬럼으로부터 요일을 뽑아냄.

In [6]:
train['weekday'] = train['date'].dt.weekday

weekday에 대한 one hot encoding을 실행.

In [7]:
train = pd.get_dummies(train, columns=['weekday'])

In [8]:
test['date'] = pd.to_datetime(test['date'])
test['weekday'] = test['date'].dt.weekday
test = pd.get_dummies(test, columns=['weekday'])

In [9]:
train['in_out'].value_counts()

시내    408500
시외      6923
Name: in_out, dtype: int64

In [10]:
train['in_out'] = train['in_out'].map({'시내':0, '시외':1})

In [11]:
test['in_out'] = test['in_out'].map({'시내':0, '시외':1})

## 좌표 데이터를 이용한 Feature Engineering
제주도의 인구는 서귀포시와 제주시에 몰려 있는 경우가 많다.

그러므로 해당 지역과의 거리를 각각 feature로 추가함.

dis_jejusi: 버스정류장과 제주시와의 거리

dis_seoquipo: 버스정류장과 서귀포시와의 거리

제주시의 위.경도: 33.500770, 126.522761

서귀포시의 위.경도: 33.259429, 126.558217

2개의 좌표 간의 거리를 구하는 함수는 파이썬의 geopy 패키지에 존재한다.

In [12]:
!pip install geopy

In [13]:
import geopy.distance

In [14]:
coords_jejusi = (33.500770, 126.522761)
coords_seoquipo = (33.259429, 126.558217)

In [15]:
train['dis_jejusi'] = [geopy.distance.vincenty((train['latitude'].iloc[i], train['longitude'].iloc[i]), coords_jejusi).km  for i in range(len(train))]
train['dis_seoquipo'] = [geopy.distance.vincenty((train['latitude'].iloc[i], train['longitude'].iloc[i]), coords_seoquipo).km  for i in range(len(train))]

In [16]:
train['dis_jejusi'].corr(train['18~20_ride'])

-0.12380548460763058

In [17]:
train['dis_seoquipo'].corr(train['18~20_ride'])

0.022087013666358277

제주시와의 거리가 멀면 멀수록 승차인원이 줄어드는 경향을 보이나

서귀포시와의 거리는 모호한 상관성을 띄고 있다.

In [18]:
test['dis_jejusi'] = [geopy.distance.vincenty((test['latitude'].iloc[i], test['longitude'].iloc[i]), coords_jejusi).km  for i in range(len(test))]
test['dis_seoquipo'] = [geopy.distance.vincenty((test['latitude'].iloc[i], test['longitude'].iloc[i]), coords_seoquipo).km  for i in range(len(test))]

## RandomForest을 활용한 modeling
sklearn 내부의 randomforest 함수를 이용한 모델링을 진행.

머신러닝 학습에 활용할 input 변수와 target 변수를 정의.



In [19]:
input_var=['in_out','latitude','longitude','6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride',
       '10~11_ride', '11~12_ride', '6~7_takeoff', '7~8_takeoff', '8~9_takeoff',
       '9~10_takeoff', '10~11_takeoff', '11~12_takeoff','weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
       'weekday_5', 'weekday_6', 'dis_jejusi', 'dis_seoquipo']
target=['18~20_ride']

In [20]:
X_train = train[input_var]
y_train = train[target]
X_test = test[input_var]

In [21]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state = 1217)

In [22]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1217, verbose=0, warm_start=False)

In [24]:
test['18~20_ride'] = rf.predict(X_test)
test[['id', '18~20_ride']].to_csv("dacon_base_line_easy.csv", index=False)